In [1]:
# Importing Essential Libraries
import os
import cv2
import glob
import random
from tqdm import tqdm
import math

In [2]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import cv2  # Assuming you have OpenCV installed
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
import torchvision
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import itertools

from vit_pytorch.efficient import ViT

In [3]:
train_path = "C:\\Study\\OJT\\Dataset_Extraction\\Vit2\\Train"
test_path ="C:\\Study\\OJT\\Dataset_Extraction\\Vit2\\Test"  

In [4]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

In [5]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=27,
    transformer=efficient_transformer,
    channels=3,
)

In [6]:
# Training settings
batch_size = 64
epochs = 20
lr = 0.001
gamma = 0.7
seed = 42

In [7]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adamax(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [8]:
# Define the transformation for loading and preprocessing images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),  # You can adjust the crop size as needed
    transforms.ToTensor(),
])

In [9]:
# Create a custom dataset for the entire training set
full_dataset = ImageFolder(train_path, transform=transform)

In [10]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [11]:
# Create custom datasets for training and test
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

In [12]:
# Create data loaders for training and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [13]:
# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if batch_idx % 10 == 9:  # Print every 10 batches
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {running_loss / 10:.4f}, Training Accuracy: {100 * correct / total:.2f}%")
            running_loss = 0.0

    scheduler.step()

    # Validation accuracy calculation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}, Training Accuracy: {100 * correct / total:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

# Evaluation on the test set
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()

print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")
torch.save(model.state_dict(), 'ViT_RGB_pth')

Epoch 1, Batch 10/103, Loss: 3.4608, Training Accuracy: 4.53%
Epoch 1, Batch 20/103, Loss: 3.3385, Training Accuracy: 4.53%
Epoch 1, Batch 30/103, Loss: 3.3147, Training Accuracy: 4.53%
Epoch 1, Batch 40/103, Loss: 3.3044, Training Accuracy: 4.45%
Epoch 1, Batch 50/103, Loss: 3.3105, Training Accuracy: 4.25%
Epoch 1, Batch 60/103, Loss: 3.2835, Training Accuracy: 4.32%
Epoch 1, Batch 70/103, Loss: 3.2826, Training Accuracy: 4.17%
Epoch 1, Batch 80/103, Loss: 3.2042, Training Accuracy: 4.86%
Epoch 1, Batch 90/103, Loss: 3.0763, Training Accuracy: 5.54%
Epoch 1, Batch 100/103, Loss: 2.9570, Training Accuracy: 6.27%
Epoch 1, Training Accuracy: 6.24%, Validation Accuracy: 11.47%
Epoch 2, Batch 10/103, Loss: 2.8264, Training Accuracy: 13.12%
Epoch 2, Batch 20/103, Loss: 2.8209, Training Accuracy: 14.06%
Epoch 2, Batch 30/103, Loss: 2.8154, Training Accuracy: 14.11%
Epoch 2, Batch 40/103, Loss: 2.7130, Training Accuracy: 14.22%
Epoch 2, Batch 50/103, Loss: 2.6456, Training Accuracy: 14.69%
E